In [49]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
from scipy.io import wavfile
from collections import defaultdict, Counter
from scipy import signal
import numpy as np
import librosa
import random as rn
from keras.layers import Dense
from keras import Input
#from keras.engine import Model
from keras.layers import Dense, TimeDistributed, Dropout, Bidirectional, GRU, BatchNormalization, Activation, LeakyReLU, LSTM, Flatten, RepeatVector, Permute, Multiply, Conv2D, MaxPooling2D
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
import csv 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import keras
from keras import layers

# data_dir = '/content/drive/MyDrive/Dataset/audiofile_shorted500/'
volume_path = 'C:/Users/GJ/Desktop/연구실/2022SPCUP/WAV_Ordered train set'
# volume_path = 'C:/Users/GJ/Desktop/연구실/2022SPCUP/spcup_2022_training_part1'


FilePathList =[]

for dirName, subdirList, fileList in os.walk(volume_path):
    for filename in fileList:
        if '.csv' not in filename:
            # print(filename)
            FilePathList.append(volume_path+'/'+filename)
        else:
            pd_label = pd.read_csv(volume_path+'/'+filename)

labels = np.array(pd_label['algorithm'])



In [50]:
##파일마다 1초씩 슬라이싱 해서 data augmentation
# X_aug = []

sampling_rate = 16000

y_labels_aug = []
X_mel_aug = []
for i, filepath in enumerate(FilePathList):
    # fig, ax = plt.subplots()
    y, sr = librosa.load(filepath, mono=True, sr = sampling_rate)
    index_f = 0
    for j in range(y.size//sampling_rate):
        X_aug_seg = y[index_f:index_f+sampling_rate]

        
        S = librosa.feature.melspectrogram(y=X_aug_seg, sr=sampling_rate, n_mels=128, fmax=8000)
        S_dB = librosa.power_to_db(S, ref=np.max)
        X_mel_aug.append(S_dB)

        y_labels_aug.append(labels[i])
    
        index_f = index_f+sampling_rate
        #         X_aug_seg = np.expand_dims(X_aug_seg, axis=0)
        #         X_aug.append(X_aug_seg)
# X_aug = np.concatenate(X_aug,axis = 0)
X_spec_aug = np.array(X_spec_aug)
y_labels_aug = np.array(y_labels_aug)
print(X_spec_aug.shape)
print(y_labels_aug.shape)

X = X_spec_aug
Y = y_labels_aug


In [74]:
##파일마다 1초씩 슬라이싱 해서 data augmentation + 마지막에 파일 별로 score 계산하기 위한 preprocess
# X_aug = []

sampling_rate = 16000
n_mels = 64

X_train_path, X_test_path, y_train_raw, y_test_raw = train_test_split(np.array(FilePathList), labels, test_size=0.2, stratify = labels, random_state=42)

##train set
print('raw train_set_num :',len(y_train_raw))

y_labels_aug = []
X_mel_aug = []
for i, filepath in enumerate(X_train_path):
    # fig, ax = plt.subplots()
    y, sr = librosa.load(filepath, mono=True, sr = sampling_rate)
    index_f = 0
    for j in range(y.size//sampling_rate):
        X_aug_seg = y[index_f:index_f+sampling_rate]

        
        S = librosa.feature.melspectrogram(y=X_aug_seg, sr=sampling_rate, n_mels=n_mels, fmax=8000)
        S_dB = librosa.power_to_db(S, ref=np.max)
        X_mel_aug.append(S_dB)

        y_labels_aug.append(y_train_raw[i])
    
        index_f = index_f+sampling_rate
        #         X_aug_seg = np.expand_dims(X_aug_seg, axis=0)
        #         X_aug.append(X_aug_seg)
# X_aug = np.concatenate(X_aug,axis = 0)
X_train1 = np.array(X_mel_aug)
y_train1 = np.array(y_labels_aug)
print('train set.shape : ',X_train1.shape)
print('y_train.shape',y_train1.shape )



raw train_set_num : 4000
train set.shape :  (25875, 64, 32)
y_train.shape (25875,)


In [75]:
##test set
print('raw test_set_num :',len(y_test_raw))
y_labels_aug = []
X_mel_aug = []
file_split_num = []
for i, filepath in enumerate(X_test_path):
    # fig, ax = plt.subplots()
    y, sr = librosa.load(filepath, mono=True, sr = sampling_rate)
    index_f = 0
    for j in range(y.size//sampling_rate):
        X_aug_seg = y[index_f:index_f+sampling_rate]

        
        S = librosa.feature.melspectrogram(y=X_aug_seg, sr=sampling_rate, n_mels=n_mels, fmax=8000)
        S_dB = librosa.power_to_db(S, ref=np.max)
        X_mel_aug.append(S_dB)

        y_labels_aug.append(y_test_raw[i])
    
        index_f = index_f+sampling_rate
    file_split_num.append(j+1)
        
X_test1 = np.array(X_mel_aug)
y_test1 = np.array(y_labels_aug)
print('splited_test set.shape : ',X_test1.shape)
print('y_test.shape : ', y_test1.shape)
print(len(file_split_num))

raw test_set_num : 1000
splited_test set.shape :  (6423, 64, 32)
y_test.shape :  (6423,)
1000


In [78]:
###Train ,Test data split
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# X_train, X_test, y_train, y_test = train_test_split(pad_x_arr, label, test_size=0.2, stratify = label, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify = Y, random_state=42)

print('np.unique(y_train)',np.unique(y_train1))

for i in range(5):
    print('전체 데이터 label'+'==',i, '분포 : ', labels.tolist().count(i))
for i in range(5):
    print('train set label'+'==',i, '분포 : ', y_train1.tolist().count(i)/labels.tolist().count(i))
for i in range(5):
    print('test set label'+'==',i, '분포 : ', y_test1.tolist().count(i))
    
X_train_ = X_train1.reshape(X_train1.shape[0],X_train1.shape[1],X_train1.shape[2],1)
X_test = X_test1.reshape(X_test1.shape[0],X_test1.shape[1],X_test1.shape[2],1)
print('X_train.shape : ',X_train.shape)
print('X_test.shape : ',X_test.shape)

y_train =to_categorical(y_train1) 
y_test = to_categorical(y_test1) 
# print(y_test.shape)
print('y_train.shape',y_train.shape)
print('y_test.shape : ',y_test.shape)

# print('y : ',y_test[:10])

np.unique(y_train) [0 1 2 3 4]
전체 데이터 label== 0 분포 :  1000
전체 데이터 label== 1 분포 :  1000
전체 데이터 label== 2 분포 :  1000
전체 데이터 label== 3 분포 :  1000
전체 데이터 label== 4 분포 :  1000
train set label== 0 분포 :  6.252
train set label== 1 분포 :  4.779
train set label== 2 분포 :  4.634
train set label== 3 분포 :  6.127
train set label== 4 분포 :  4.083
test set label== 0 분포 :  1511
test set label== 1 분포 :  1150
test set label== 2 분포 :  1219
test set label== 3 분포 :  1513
test set label== 4 분포 :  1030
X_train.shape :  (25875, 64, 32)
X_test.shape :  (6423, 64, 32, 1)
y_train.shape (25875, 5)
y_test.shape :  (6423, 5)


In [86]:
## model compile & fit

from keras.models import Sequential
from keras.layers import Flatten, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint


model = Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(X_train_.shape[1:])))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dense(32, activation='relu'))
model.add(Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



early_stopping = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 1, mode = 'auto',restore_best_weights=True)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True) #최상 가중치 저장

classifier = model.fit(X_train,
                    y_train,
                    epochs=50,
                    batch_size=64, callbacks = [early_stopping],
                    validation_split = 0.2)

plt.plot(classifier.history['accuracy'])
plt.plot(classifier.history['val_accuracy'])
# plt.plot(classifier.history['loss'])
# plt.plot(classifier.history['val_loss'])
plt.legend(['val_accuracy', 'accuracy'], loc = 'upper left')
plt.show()

test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('테스트 정확도:', test_acc)

prediction = model.predict(X_test)

Epoch 1/50
324/324 [==============================] - 15s 44ms/step - loss: 1.9694 - accuracy: 0.5256 - val_loss: 0.8235 - val_accuracy: 0.7161
Epoch 2/50
324/324 [==============================] - 14s 43ms/step - loss: 0.6232 - accuracy: 0.7871 - val_loss: 0.5424 - val_accuracy: 0.8139
Epoch 3/50
 29/324 [=>............................] - ETA: 11s - loss: 0.5191 - accuracy: 0.8125

KeyboardInterrupt: 

In [83]:
from scipy.stats import mode
predicted_classes = np.argmax(prediction, axis = 1)
f = 0
test_predict=[]
for i in file_split_num:
#     print(predicted_classes[f:f+i])
    test_predict.append(mode(predicted_classes[f:f+i])[0][0])
    f+=i
                    
acc_bool = test_predict==y_test_raw
print(test_predict[:100])
print(y_test_raw[:100])
print(acc_bool[:100])
test_accuracy = acc_bool.tolist().count(True)/len(acc_bool)
print(test_accuracy)
print('test_set.size : ', acc_bool.size)

[1, 3, 0, 2, 2, 0, 0, 1, 0, 2, 2, 1, 0, 0, 2, 4, 0, 2, 1, 3, 4, 2, 4, 4, 0, 3, 1, 2, 0, 3, 4, 3, 1, 1, 0, 3, 0, 2, 2, 4, 0, 0, 3, 2, 0, 2, 3, 4, 1, 3, 1, 1, 3, 1, 2, 3, 3, 0, 3, 3, 1, 0, 2, 0, 1, 3, 4, 3, 3, 1, 4, 1, 0, 2, 0, 0, 4, 1, 4, 0, 0, 2, 2, 4, 1, 0, 3, 2, 1, 4, 3, 2, 2, 3, 2, 3, 2, 4, 1, 3]
[1 3 0 2 2 0 0 1 0 2 2 1 0 0 2 4 0 2 1 3 4 2 4 4 0 3 2 2 0 3 4 3 2 2 0 3 0
 2 2 4 0 0 3 2 0 2 3 4 1 3 2 1 3 1 2 3 3 0 3 3 1 0 1 0 1 3 4 3 3 2 4 1 0 2
 0 0 4 1 4 0 0 2 2 4 1 0 3 1 1 4 3 2 1 3 2 3 2 4 1 3]
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False  True  True  True  True  True False False  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True False  True  True  True  True  True  True  True  True  True
  True  True False  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True  True  T